### *Assessment of pathways using the IAMC 1.5°C Scenario Data*

<img style="float: right; height: 100px; margin-top: 10px;" src="../_static/IIASA_logo.png">
<img style="float: right; height: 80px;" src="../_static/IAMC_logo.jpg">

# Energy intensity in mitigation pathways

This notebook computes the energy intensity indicators and plots a number of figures for all pathways and for pathways limiting global warming to below 1.5°C by the end of the century, with focus on the four illustrative pathways.

The scenario data used in this analysis can be accessed and downloaded at [https://data.ene.iiasa.ac.at/iamc-1.5c-explorer](https://data.ene.iiasa.ac.at/iamc-1.5c-explorer).

## Load ``pyam`` package and other dependences

In [ ]:
import pandas as pd
import numpy as np
import warnings
import io
import itertools
import yaml
import math
import matplotlib.pyplot as plt
plt.style.use('../assessment/style_sr15.mplstyle')
%matplotlib inline
import pyam

## Import scenario data, categorization and specifications files

The metadata file must be generated from the notebook `sr15_2.0_categories_indicators`
in the repository https://github.com/iiasa/ipcc_sr15_scenario_analysis.
If the snapshot file has been updated, make sure that you rerun the categorization notebook.

The last cell of this section loads and assigns a number of auxiliary lists as defined in the categorization notebook.

In [ ]:
sr1p5 = pyam.IamDataFrame(data='../data/iamc15_scenario_data_world_r1.xlsx')

In [ ]:
sr1p5.load_metadata('../assessment/sr15_metadata_indicators.xlsx')

In [ ]:
with open("../assessment/sr15_specs.yaml", 'r') as stream:
    specs = yaml.load(stream)

rc = pyam.run_control()
for item in specs.pop('run_control').items():
    rc.update({item[0]: item[1]})
cats = specs.pop('cats')
cats_15 = specs.pop('cats_15')
cats_15_no_lo = specs.pop('cats_15_no_lo')
cats_2 = specs.pop('cats_2')
marker = specs.pop('marker')

## Downselect scenario ensemble to categories of interest for this assessment

In [ ]:
df = sr1p5.filter(category=cats, year=range(2010, 2101, 10))

## Initialize a `pyam.Statistics` instance

In [ ]:
stats = pyam.Statistics(df=df, groupby={'marker': ['LED', 'S1', 'S2', 'S5']},
                        filters=[(('pathways', 'no & lo os 1.5c'), {'category': cats_15_no_lo}),
                                 (('pathways', 'high os 1.5c'), {'category': '1.5C high overshoot'}),
                                 (('pathways', 'lower 2c'), {'category': 'Lower 2C'}),
                                 (('pathways', 'higher 2c'), {'category': 'Higher 2C'})],
                       rows=True)

## Computing energy intensity indicators

In [ ]:
fe = df.filter(variable='Final Energy').timeseries()
fe.index = fe.index.droplevel([2, 3, 4])

In [ ]:
pe = df.filter(variable='Primary Energy').timeseries()
pe.index = pe.index.droplevel([2, 3, 4])

In [ ]:
gdp = df.filter(variable='GDP|PPP').timeseries()
gdp.index = gdp.index.droplevel([2, 3, 4])

In [ ]:
primary_energy_intensity = pe / gdp * 1000
final_energy_intensity = fe / gdp * 1000

In [ ]:
stats.add(primary_energy_intensity, 'Energy intensity (EJ/million US$2010)', row='Primary energy')
stats.add(final_energy_intensity, 'Energy intensity (EJ/million US$2010)', row='Final energy')

In [ ]:
for y in [2030, 2050, 2100]:
    stats.add((primary_energy_intensity[y] / primary_energy_intensity[2010] - 1) * 100,
        'change in energy intensity (% relative to 2010)',
        subheader=y, row='Primary energy')
    stats.add((final_energy_intensity[y] / final_energy_intensity[2010] - 1) * 100,
        'change in energy intensity (% relative to 2010)',
        subheader=y, row='Final energy')

In [ ]:
summary = stats.summarize()
summary

In [ ]:
summary.to_excel('iamc15_energy_intensity.xlsx')